In [1]:
import os, time

In [2]:
%run ../utilities/postgres_utilities.ipynb

In [3]:
#embedding_endpoint =  os.environ.get('embedding_model_endpoint')
model_endpoint =  'https://llama-31-8b-instruct-samouelian-edb-ai.apps.ai-dev01.kni.syseng.devcluster.openshift.com/v1/chat/completions'
model_name = 'Llama-31-8b-instruct'
number_data_rows = 100

In [4]:
connect_db()

In [5]:
sql = """
    select aidb.delete_model('llama-model');
    """

In [6]:
execute_sql(sql)

In [7]:
sql = f"""
    select aidb.create_model(
    'llama-model',
    'openai_completions',
    '{{"model":"llama-31-8b-instruct", "url":"{model_endpoint}", 
    "dimensions":8096}}'::JSONB,
    '{{"api_key":""}}'::JSONB);
    """

In [8]:
execute_sql(sql)

In [9]:
#Compute time for 1k embeddings

In [10]:
sql = f"select description from film limit {number_data_rows}"
results = execute_sql_results(sql)

In [11]:
encode_text_sql = "select aidb.decode_text('llama-model', %s)"

In [14]:
question_response_pairs = list()
test_description = "Starting online generation test using edb and default vllm configuration"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Text output generated ...", end=" ")
for index, film in enumerate(results):
    description = f"Briefly expand this movie description: {film[0]}"
    generated_text = execute_sql_results_params(encode_text_sql, description)
    question_response_pairs.append(f"{description}:{generated_text}")
    if index % 100 == 0:
        print(f"{index}", end=" ")
toc = time.perf_counter()
print(f"{index+1}")
print(f"Time to compute {number_data_rows} text generation tasks: {toc - tic:0.4f} seconds")
print(len(test_description) * "-")

Starting online generation test using edb and default vllm configuration
------------------------------------------------------------------------
Text output generated ... 0 3
Time to compute 3 text generation tasks: 26.8923 seconds
------------------------------------------------------------------------


In [ ]:
close_db()